In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../train/pooled_df.pkl')
df.head()

,SUBJECT_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,...,diagnosisstring,subject_id,stay_id,first_careunit,last_careunit,intime,outtime,los,anchor_age,dod
0,1027140.0,3011529.0,carevue,CCU,CCU,57.0,57.0,2152-07-23 05:34:12,2152-07-24 18:37:32,1.5440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
1,1027482.0,3011175.0,carevue,MICU,MICU,12.0,12.0,2162-03-20 17:22:48,2162-03-22 18:26:10,2.0440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
2,1011746.0,3013234.0,carevue,MICU,MICU,52.0,52.0,2107-07-28 17:22:31,2107-08-01 11:03:44,3.7370,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
3,1010325.0,3022526.0,metavision,MICU,MICU,52.0,52.0,2145-02-06 02:15:08,2145-02-07 18:47:37,1.6892,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
4,1012708.0,3035542.0,metavision,SICU,SICU,33.0,33.0,2175-02-20 07:41:28,2175-02-23 10:21:01,3.1108,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT


In [2]:
print(df.columns)

Index(['SUBJECT_ID', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG',
       'age', 'readmission', 'mortality', 'los_3day', 'los_7day', 'ICD9_CODE',
       '12h_obs', '24h_obs', 'ID', 'code_name', 'code_offset', 'value', 'uom',
       'code_order', 'seq_len', 'patienthealthsystemstayid', 'gender',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


# Load the pre-trained model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed(input_text):
    tokens = tokenizer(input_text, return_tensors="pt")
    embeddings = model(**tokens).last_hidden_state
    pooled_embeddings = embeddings.mean(dim=1)
    return pooled_embeddings

/Users/noelkj/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you ex

In [5]:
def embed_data(df):
    # Create empty numpy array with len(df) rows and 256 columns
    df_embed = []
    for k in range(len(df)):
        df_embed.append([])
        for n in range(len(df['code_name'][k])):
            input_text = 'code_name ' + str(df['code_name'][k][n]) + ' value ' + str(df['value'][k][n]) + ' uom ' + str(df['uom'][k][n])
            print(input_text)
            embed_input = embed(input_text)
            print(embed_input.shape)
            # Save to df
            df_embed[k].append(embed_input)
    return pd.DataFrame(df_embed)

# Test with first 10 rows
df_test = df.head(2)
df_test = embed_data(df_test)
df_test.head()

# Save to pickle
#df_test.to_pickle('df_test.pkl')

code_name Anion Gap value 11.0 uom mEq/L
torch.Size([1, 768])
code_name Bicarbonate value 25.0 uom mEq/L
torch.Size([1, 768])
code_name Bilirubin, Total value 0.3 uom mg/dL
torch.Size([1, 768])
code_name Cholesterol Ratio (Total/HDL) value 5.9 uom Ratio
torch.Size([1, 768])
code_name Cholesterol, HDL value 31.0 uom mg/dL
torch.Size([1, 768])
code_name Cholesterol, LDL, Calculated value 108.0 uom mg/dL
torch.Size([1, 768])
code_name Cholesterol, Total value 183.0 uom mg/dL
torch.Size([1, 768])
code_name Creatinine value 0.7 uom mg/dL
torch.Size([1, 768])
code_name Magnesium value 1.7 uom mg/dL
torch.Size([1, 768])
code_name Potassium value 4.1 uom mEq/L
torch.Size([1, 768])
code_name Sodium value 143.0 uom mEq/L
torch.Size([1, 768])
code_name Urea Nitrogen value 13.0 uom mg/dL
torch.Size([1, 768])
code_name INR(PT) value 1.0 uom  
torch.Size([1, 768])
code_name MCHC value 34.4 uom %
torch.Size([1, 768])
code_name MCV value 93.0 uom fL
torch.Size([1, 768])
code_name Platelet Count value 

KeyboardInterrupt: 

: 